<a href="https://colab.research.google.com/github/leenasilvoster/Mytest/blob/master/CNN_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D
from keras.layers import LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint 
from google.colab import files

In [ ]:
#fix random seed for reproducibility
np.random.seed(7)
from prettytable import PrettyTable
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Load the dataset but only keep the top n words,zero the rest
top_words=10000
(X_train,y_train),(x_test,y_test) = imdb.load_data(num_words=top_words)

17473536/17464789 [==============================] - 0s 0us/step


In [ ]:
#num_words = top_words
print("Shape of train data:",X_train.shape)
X_train,x_cv,y_train,y_cv = train_test_split(X_train,y_train,test_size=0.2)
print("Shape of train data:",X_train.shape)
print("Shape of test data:",x_test.shape)
print("Shape of cv data:",x_cv.shape)

Shape of train data: (25000,)
Shape of train data: (20000,)
Shape of test data: (25000,)
Shape of cv data: (5000,)


In [ ]:
#truncate and pad input sequence
max_review_length = 600
X_train = sequence.pad_sequences(X_train,maxlen = max_review_length )
x_test = sequence.pad_sequences(x_test,maxlen = max_review_length )
x_cv = sequence.pad_sequences(x_cv,maxlen = max_review_length )


In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words,embedding_vecor_length,input_length = max_review_length))

In [ ]:
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 600, 32)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#filepath ="C:\\Users\\princ\\Documents\\weights_best.hdfs"
filepath ="/content/drive/MyDrive/Colab Notebooks/weights_best.hdfs"
checkpoint = ModelCheckpoint(filepath,monitor='val_acc', verbose=1,save_best_only=True,mode='max',save_weights_only=True)
callbacks_list =[checkpoint]
model.fit(X_train,y_train,epochs=1,batch_size=256,verbose=1, callbacks=callbacks_list,validation_data=(x_cv,y_cv))
#model.fit(X_train,y_train,batch_size=256,epochs=5,verbose=1,callbacks=callbacks_list,validation_data=(x_cv,y_cv))
#For weights
model.save('weights_best.hdfs')
files.download('weights_best.hdfs')

79/79 [==============================] - 292s 3s/step - loss: 0.6855 - accuracy: 0.5410 - val_loss: 0.5849 - val_accuracy: 0.7006
INFO:tensorflow:Assets written to: weights_best.hdfs/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#final evaluation of the model
embedding_vector_length = 32
model =Sequential()
model.add(Embedding(top_words,embedding_vector_length,input_length = max_review_length))
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.load_weights("weights_best.hdfs")
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 600, 32)           320000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
scores = model.evaluate(x_test,y_test,verbose=1,batch_size=256)
print("Accuracy:%.2f" %(scores[1]*100))

98/98 [==============================] - 54s 554ms/step - loss: 0.5858 - accuracy: 0.7038
Accuracy:70.38


Accuracy:70.38
